In [ ]:
import numpy as np
import gym
import d4rl

env = gym.make("walker2d-medium-v2")
dataset = env.get_dataset()

In [ ]:
dataset["next_observations"].shape

env.observation_space, env.action_space

In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium.wrappers.env_checker import PassiveEnvChecker
from gymnasium.wrappers.order_enforcing import OrderEnforcing
from gymnasium.wrappers.time_limit import TimeLimit
from gymnasium.envs.mujoco.walker2d_v4 import Walker2dEnv


official_env = gym.make("Walker2d-v4")
unofficial_env = TimeLimit(OrderEnforcing(PassiveEnvChecker(
    Walker2dEnv(exclude_current_positions_from_observation=False)
)), max_episode_steps=1000)

In [ ]:
state, info = unofficial_env.reset()
action = np.zeros_like(unofficial_env.action_space.low)
next_state, reward, term, trun, info = unofficial_env.step(action)

In [ ]:
(- np.sum(np.square(action)) * unofficial_env.env.env.env._ctrl_cost_weight
 + 1.0
 + (next_state[0] - state[0]) / unofficial_env.env.env.env.dt * unofficial_env.env.env.env._forward_reward_weight)

In [ ]:
reward

## Run PPO agent to collect dataset

- Train PPO agent
- Use "half" trained PPO agent and collect a dataset



In [ ]:
from typing import List
from itertools import product
from functools import partial
import os
import multiprocessing
import gymnasium as gym
import numpy as np

from io_agent.control.ppo import PPoController

from common import run_mpc, run_io_mpc
from utils import parallelize, steady_state_cost, save_experiment


general_seed = 42
ppo_path = f"/mnt/DEPO/tok/sl-to-rl/walker/ppo-3m-{general_seed}"

if not os.path.exists(".".join([ppo_path, "zip"])):
    PPoController.train(lambda: gym.make("Walker2d-v4"),
                        n_envs=16,
                        seed=general_seed,
                        path=ppo_path,
                        total_timesteps=int(3e6),
                        ppo_kwargs=dict(
                            ent_coef=1e-4,
                            learning_rate=3e-4),
                        )
ppo_agent = PPoController(ppo_path)

In [ ]:
from typing import List, Dict, Union, Any, Callable, Tuple, Type
from functools import partial
import numpy as np
import gym
import d4rl

from io_agent.plant.base import NominalLinearEnvParams, LinearConstraint, LinearConstraints
from io_agent.evaluator import ControlLoop, Transition
from io_agent.control.mpc import MPC
from io_agent.control.rmpc import RobustMPC
from io_agent.control.io import IOController, AugmentedTransition, AugmentDataset
from io_agent.utils import FeatureHandler


env = gym.make("walker2d-medium-v2")
dataset = env.get_dataset()

# nominal_model = NominalLinearEnvParams(
#     matrices=None,
#     constraints=LinearConstraints(
#         action=LinearConstraint(
#             matrix=np.block([
#                 [np.eye(env.action_space.shape[0])],
#                 [-np.eye(env.action_space.shape[0])]]),
#             vector=np.concatenate([
#                 env.action_space.high,
#                 -env.action_space.low
#             ])
#         )
#     ),
#     costs=None
# )

# feature_handler = FeatureHandler(
#     params=nominal_model,
#     n_past=0,
#     add_bias=False,
#     use_action_regressor=False,
#     use_noise_regressor=False,
#     use_state_regressor=False,
#     noise_size=0,
#     state_size=env.observation_space.shape[0],
#     action_size=env.action_space.shape[0],
#     output_size=env.observation_space.shape[0],
# )
# augmenter = AugmentDataset(
#     expert_agent=None,
#     feature_handler=feature_handler
# )


# general_seed = 42
# rng = np.random.default_rng(general_seed)
# indices = rng.integer(0, len(dataset["observations"]), size = 1000)

# trajectory = [Transition(
#     state=dataset["observations"][index],
#     next_state=dataset["next_observations"][index],
#     action=dataset["actions"][index],
#     reward=dataset["observation"][index],
#     state=dataset["observation"][index],
#     state=dataset["observation"][index],
# ) for index in indices]
# augmenter([])

- - -
## Iterative IO algorithm
- - -
#### Quadrotor Environment

The Q minimization:
$$
\min_u 2s^T\theta_{su}u + u^T\theta_{uu}u \\ \text{s.t.} \quad G_u u \leq h_u
$$

- Create dataset

In [ ]:
from typing import List
from itertools import product
from functools import partial
import numpy as np
import multiprocessing

from io_agent.evaluator import Transition
from io_agent.plant.quadrotor import QuadrotorEnv

from common import run_mpc, run_io_rmpc
from utils import parallelize, steady_state_cost, save_experiment


n_cpu = multiprocessing.cpu_count()
horizon = 25
n_trials = 20
n_past = 2
add_bias = False
n_dataset_trials = 20
n_rhos = 12
general_seed = 42
seed_rng = np.random.default_rng(general_seed)

plant = QuadrotorEnv()
permute_seed, *trial_seeds = seed_rng.integers(0, 2**30, n_trials + 1)
dataset_trial_seeds = seed_rng.integers(0, 2**30, n_dataset_trials)

dataset_trajectories = parallelize(
    n_proc=min(n_cpu, n_dataset_trials),
    fn=partial(run_mpc, plant=plant),
    kwargs_list=[
        dict(
            horizon=horizon,
            use_foresight=False,  # Without hindsight data
            bias_aware=False,
            env_reset_rng=np.random.default_rng(_seed)
        ) for _seed in dataset_trial_seeds
    ],
    loading_bar_kwargs=dict(desc="MPC dataset trials")
)

save_experiment(
    values={"mpc_dataset": dataset_trajectories},
    seed=general_seed,
    exp_dir="./quadrotor_data/dataset",
    name="mpc_obl"
)

- Construct the Q function

In [1]:
from typing import Callable, List, Tuple, Optional, Any, Union, Dict
from dataclasses import dataclass
from functools import partial
from tqdm.notebook import tqdm
import numpy as np
import torch
import geotorch
import cvxpy as cp

from io_agent.plant.quadrotor import QuadrotorEnv
from io_agent.plant.base import (NominalLinearEnvParams,
                                 LinearizationWrapper,
                                 LinearConstraints,
                                 LinearConstraint,
                                 Plant)
from io_agent.control.mpc import Optimizer
from io_agent.utils import AugmentedTransition, FeatureHandler
from io_agent.control.io import AugmentDataset
from io_agent.control.mpc import MPC
from io_agent.control.deep_io import IterativeIOController
from io_agent.evaluator import Transition

from utils import load_experiment, parallelize
from common import run_agent


@dataclass
class DeepIOArgs():
    n_past: int = 2
    add_bias: int = False
    use_action_regressor: bool = False
    use_noise_regressor: bool = True
    use_state_regressor: bool = False
    horizon: int = 25
    use_expert: bool = False
    learning_rate: float = 1e-2
    lr_exp_decay: float = 0.98
    n_epoch: int = 1000
    n_batch: int = 128


def prepare_deep_io(dataset: List[List[Transition]],
                  env: Plant,
                  rng: np.random.Generator,
                  args: DeepIOArgs,
                  verbose: bool = True
                  ) -> IterativeIOController:
    feature_handler = FeatureHandler(
        params=env.nominal_model(),
        n_past=args.n_past,
        add_bias=args.add_bias,
        use_action_regressor=args.use_action_regressor,
        use_noise_regressor=args.use_noise_regressor,
        use_state_regressor=args.use_state_regressor,
    )
    expert_agent = None
    if args.use_expert:
        expert_agent = MPC(
            action_size=lin_env.action_size,
            state_size=lin_env.state_size,
            noise_size=lin_env.noise_size,
            output_size=lin_env.output_size,
            horizon=args.horizon)
        expert_agent.optimizer = expert_agent.prepare_optimizer(feature_handler.params)
    augmenter = AugmentDataset(
        expert_agent=expert_agent,
        feature_handler=feature_handler,
    )
    augmented_dataset = augmenter(dataset)
    torch.manual_seed(rng.integers(0, 2**30).item())
    iterative_io_agent = IterativeIOController(
        constraints=feature_handler.params.constraints,
        feature_handler=feature_handler,
        learning_rate=args.learning_rate,
        include_constraints=True,
        action_constraints_flag=True,
        state_constraints_flag=False,
        lr_exp_decay=args.lr_exp_decay,
    )
    return iterative_io_agent, augmented_dataset

In [5]:
import multiprocessing

from common import run_agent
from utils import save_experiment


lin_env = LinearizationWrapper(QuadrotorEnv())
dataset = load_experiment("./quadrotor_data/dataset/mpc_obl-42")["mpc_dataset"]

n_trials = 50
n_cpu = multiprocessing.cpu_count()
general_seed = 43
seed_rng = np.random.default_rng(general_seed)
trial_seeds = seed_rng.integers(0, 2**30, n_trials)
evaluation_epochs = [int(val) for val in np.floor(np.logspace(1, 2, 15))]


def deep_io_trials(args: DeepIOArgs, rng: np.random.Generator, trial_seeds: List[int]):
    if args.n_epoch != 0:
        raise ValueError("```n_epoch``` must be set to 0")
    deep_io_agent, augmented_dataset = prepare_deep_io(
        dataset, lin_env, rng, args, verbose=False)
    losses = []
    costs = {}
    with tqdm(total=evaluation_epochs[-1]) as pbar:
        for eval_break_epoch in evaluation_epochs:
            print("eval_break_epoch", eval_break_epoch)
            n_epoch = eval_break_epoch - len(losses)
            _losses = deep_io_agent.train(
                augmented_dataset,
                epochs=n_epoch,
                batch_size=args.n_batch,
                rng=rng,
                verbose=False)
            losses.extend(_losses)

            deep_io_trajectories = parallelize(
                n_proc=min(n_cpu, 50),
                fn=partial(run_agent, agent=deep_io_agent, plant=lin_env),
                kwargs_list=[
                    dict(
                        use_foresight=False,
                        bias_aware=False,
                        env_reset_rng=np.random.default_rng(_seed)
                    ) for _seed in trial_seeds
                ],
            )
            deep_io_costs = [300 - np.sum([np.exp(-tran.cost) for tran in trajectory])
                             for trajectory in deep_io_trajectories]
            costs[eval_break_epoch] = deep_io_costs
            pbar.set_postfix({"Median cost": np.median(deep_io_costs)})
            pbar.update(n_epoch)
    return costs, losses


experiment_args = {
    "DeepIO-No-expert": DeepIOArgs(use_expert=False, n_epoch=0),
    # "DeepIO-MPC-expert": DeepIOArgs(use_expert=True, n_epoch=0, lr_exp_decay=0.97, n_batch=32),
}

results = {}
for key, args in experiment_args.items():
    results[key] = deep_io_trials(
        args,
        np.random.default_rng(seed_rng.integers(0, 2*30)),
        trial_seeds)

/mnt/DEPO/tok/miniconda3/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


  0%|          | 0/100 [00:00<?, ?it/s]

eval_break_epoch 10
(array([0.11871567, 0.13893078], dtype=float32), None)


In [ ]:

save_experiment(
    values={"deep-io": results},
    seed=general_seed,
    exp_dir="./quadrotor_data/ablation",
    name="deep-io_2"
)

In [ ]:
import numpy as np
import matplotlib.ticker as tck

from utils import steady_state_cost, load_experiment
from collections import defaultdict
from plotter import histogram_figure, histogram_figure_plt, tube_figure_plt


fig = histogram_figure_plt(
    cost_data={key: value[0][100] for key, value in results.items()},
    title="",
    bw_method="scott",
    bw_adjust=0.30,
    log_yaxis=True,
    y_label="log density",
    low_y=1e-3,
    figsize=(6, 3)
)
fig.savefig(f"figure_data/hist_deep.svg", format="svg", dpi=1200)

fig, axes = tube_figure_plt(
    cost_data={key: value[0] for key, value in results.items()},
    title=f"",
    log_xaxis=True,
    log_yaxis=False,
    x_label="epoch",
    y_label="episodic cost",
    percentiles=(20, 80)
)
fig.savefig(f"figure_data/perf_vs_epoch_deep.svg", format="svg", dpi=1200)

fig, axes = tube_figure_plt(
    cost_data={key: {index + 1: value for index, value in enumerate(value[1])} for key, value in results.items()},
    title=f"",
    log_xaxis=True,
    log_yaxis=True,
    x_label="epoch",
    y_label="sub loss",
    percentiles=(20, 80)
)
fig.savefig(f"figure_data/loss_vs_epoch_deep.svg", format="svg", dpi=1200)


In [ ]:
{key: {_key: np.median(_val) for _key, _val in value[0].items()} for key, value in results.items()}

In [ ]:
# from io_agent.control.io import IOController


# io_agent = IOController(
#     params=nominal_dyn,
#     dataset_length=300,
#     feature_handler=feature_handler,
#     include_constraints=True,
#     action_constraints_flag=True,
#     state_constraints_flag=False,
# )
# io_agent.train(
#     augmented_dataset,
#     rng=np.random.default_rng(42))
# io_agent.action_optimizer = io_agent.prepare_action_optimizer()

## [TEST] Is minimizer ?

In [ ]:
batch_indices = np.random.default_rng(42).permutation(len(dataset))[:10]
aug_states = [augmented_dataset[index].aug_state for index in batch_indices]
exp_actions = [augmented_dataset[index].action for index in batch_indices]

actions = iterative_io_agent.batch_minimizer_actions(
    aug_states=torch.from_numpy(np.stack(aug_states, axis=0)).float(),
    theta_uu=iterative_io_agent.th_theta_uu.weight,
    theta_su=iterative_io_agent.th_theta_su.weight
).detach().cpu().numpy()

index = -1
theta_uu = iterative_io_agent.th_theta_uu.weight.detach().cpu().numpy()
theta_su = iterative_io_agent.th_theta_su.weight.detach().cpu().numpy()

def q_function(act, state):
    return (act * (theta_uu @ act)).sum() + 2 * (state * (theta_su @ act)).sum()

print("Solution:", q_function(actions[index], aug_states[index]))

values = [q_function(lin_env.action_space.sample(), aug_states[index]) for _ in range(100)]
min(values)

## [TEST] Is loss?

In [ ]:
batch_indices = np.random.default_rng(42).permutation(len(augmented_dataset))[:30]
aug_states = [augmented_dataset[index].aug_state for index in batch_indices]
exp_actions = [augmented_dataset[index].action for index in batch_indices]

th_aug_states = torch.from_numpy(np.stack(aug_states, axis=0)).float()
th_exp_actions = torch.from_numpy(np.stack(exp_actions, axis=0)).float()

th_min_actions = iterative_io_agent.batch_minimizer_actions(
    aug_states=torch.from_numpy(np.stack(aug_states, axis=0)).float(),
    theta_uu=iterative_io_agent.th_theta_uu.weight,
    theta_su=iterative_io_agent.th_theta_su.weight
)

losses = iterative_io_agent.loss(th_aug_states, th_exp_actions)
th_min_actions[0], exp_actions[0]


exp_q = iterative_io_agent.q_fn(th_aug_states, th_exp_actions, iterative_io_agent.th_theta_uu.weight)
exp_q

In [ ]:
min_q = iterative_io_agent.q_fn(th_aug_states, th_min_actions, iterative_io_agent.th_theta_uu.weight)
min_q

In [ ]:
[io_cost(aug_state, exp_action) for aug_state, exp_action in zip(aug_states, exp_actions)]

In [ ]:
exp_q - min_q

In [ ]:
losses.mean()

In [ ]:
iterative_io_agent.th_theta_uu.weight.detach().numpy()

In [ ]:
io_agent.action_optimizer = io_agent.prepare_action_optimizer()
io_agent.reset()
io_agent.action_optimizer


def io_actions(aug_state):
    constraint_matrix, constraint_vector = io_agent.calculate_constraints(
        io_agent.feature_handler.original_state(aug_state))
    io_agent.action_optimizer.parameters["constraint_matrix"].value = constraint_matrix
    io_agent.action_optimizer.parameters["constraint_vector"].value = constraint_vector
    io_agent.action_optimizer.parameters["state"].value = aug_state

    solution = io_agent.action_optimizer.problem.solve()
    return io_agent.action_optimizer.variables["action"].value

def io_function(aug_state, action):
    return 2 * np.sum(aug_state * (io_agent._q_theta_su @ action)) + (action * (io_agent._q_theta_uu @ action)).sum()

def io_cost(aug_state, exp_action):
    min_action = io_actions(aug_state)
    return io_function(aug_state, exp_action) - io_function(aug_state, min_action)

io_cost(aug_states[10], exp_actions[10])


In [ ]:
from io_agent.evaluator import ControlLoop

deep_io_agent.action_optimizer = deep_io_agent.prepare_action_optimizer()
evaluator = ControlLoop(plant=lin_env, controller=deep_io_agent, rng=np.random.default_rng(42))
trajectory = evaluator.simulate(
    bias_aware=False,
    use_foresight=False,
)

np.sum([np.exp(-t.cost) for t in trajectory])

In [ ]:
batch_size = 1
states = np.stack([t.state for t in dataset["mpc_dataset"][0]][:batch_size], axis=0)


params["state"].value = states
params["theta_su"].value = theta_su.detach().numpy()
params["theta_uu"].value = theta_uu.detach().numpy()
solution = problem.solve()
solution, params["action"].value

In [ ]:
params["action"].value

In [ ]:


random_actions = np.clip(np.random.randn(1, lin_env.action_size), lin_env.action_space.low.reshape(1, -1), lin_env.action_space.high.reshape(1, -1))

np.min(2 * np.sum(states[:1, :] * (theta_su.detach().numpy() @ random_actions.T).T, axis=-1)
+ np.sum(random_actions * (theta_uu.detach().numpy() @ random_actions.T).T, axis=-1))

In [ ]:
nominal_dyn.constraints.action.matrix, nominal_dyn.constraints.action.vector

- - -
## Convex Layers
- - -

In [ ]:
import cvxpy as cp


out = cp.Parameter(shape=(10))
ws_1 = [cp.Variable(shape=(10, 10)) for _ in range(1)]
ws_2 = [cp.Variable(shape=(10, 10)) for _ in range(1)]
ws_3 = [cp.Variable(shape=(10, 10)) for _ in range(1)]

for ws in [ws_1, ws_2, ws_3]:
    logits = cp.vstack([out @ w.T for w in ws])
    out = logits
    # out = cp.max(logits, axis=0)

out





- - -
## Parametric Set of Linear Inequalities
- - -

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sympy.solvers import solve
from sympy import Symbol


g_matrix = np.array([
    [0.75, 0.25],
    [-0.25, 1],
    [-0.75, 0.25]
])

h_vector = np.array([
    0.2,
    0.2,
    0.2
])


def visualize(the_point: np.ndarray) -> None:
    plt.figure(dpi=100, figsize=(5, 4))
    x_points = np.linspace(-1.5, 1.5, 500).reshape(1, -1)
    y_points_matrix = ((h_vector.reshape(-1, 1) -
                       g_matrix[:, 0].reshape(-1, 1) * x_points) / g_matrix[:, 1].reshape(-1, 1))

    for y_points in y_points_matrix:
        plt.plot(x_points.ravel(), y_points, "k--")

    x_grid, y_grid = np.meshgrid(x_points, x_points)
    z_flat = np.all(
        g_matrix @ np.stack([x_grid.ravel(), y_grid.ravel()], axis=0) <= h_vector.reshape(-1, 1),
        axis=0)
    plt.contourf(x_grid, y_grid, z_flat.astype(np.float64).reshape(
        x_grid.shape), cmap=plt.cm.twilight, alpha=0.4)

    plt.plot(0, 0, "go", markersize=5)
    plt.plot(*the_point, "ro", markersize=5)

    plt.xlim(-1.5, 1.5)
    plt.ylim(-1.5, 1.5)

    plt.show()


visualize(np.array([0.3, 1.4]))

In [ ]:
np.linalg.eigh(g_matrix @ g_matrix.T)